<div style="text-align: left;">

## Módulo: ED-NA-001 - Extração de Dados I
<br>

## Aula 1 - Exercício 1
<br>

#### Desenvolver em grupo um ETL e um ELT a partir da extração de dados do arquivo disponível no Kaggle:
https://www.kaggle.com/datasets/kapturovalexander/spotify-data-from-pyspark-course

#### "Cada linha no conjunto de dados corresponde a uma faixa musical, com variáveis como o título, artista e ano localizadas em suas respectivas colunas. Além das variáveis fundamentais, elementos musicais de cada faixa, como o ritmo, a capacidade de dança e a tonalidade, também foram extraídos; o algoritmo para esses valores foi gerado pelo Spotify com base em uma série de parâmetros técnicos."

#### Essa é a descrição e o tipo de cada coluna do arquivo:

> #### 01. id: str, identificador da faixa.
> #### 02. nome: str, nome da faixa.
> #### 03. artistas: str, artistas da faixa.
> #### 04. duration_ms: float, duração da faixa em milissegundos.
> #### 05. release_date: data, data de lançamento da faixa.
> #### 06. year: int, ano de lançamento da faixa.
> #### 07. acousticness: float, medida de acústica da faixa.
> #### 08. danceability: float, medida de capacidade de dança da faixa.
> #### 09. energy: float, medida de energia da faixa.
> #### 10. instrumentalness: float, medida de elementos instrumentais na faixa.
> #### 11. liveness: float, medida de vivacidade da faixa.
> #### 12. loudness: float, volume da faixa.
> #### 13. speechiness: float, medida de fala na faixa.
> #### 14. tempo: float, ritmo da faixa.
> #### 15. valence: float, medida de valência (positividade) da faixa.
> #### 16. mode: int, modo da faixa (maior ou menor).
> #### 17. key: int, tonalidade da faixa.
> #### 18. popularity: int, pontuação de popularidade da faixa.
> #### 19. explicit: int, indicação da presença de conteúdo explícito (explícito ou implícito).

#### Os dados transformados precisam responder as seguintes perguntas:
> #### 1. As músicas mais populares nos últimos 10 anos;
> #### 2. A quantidade de músicas lançadas por artista e ano de lançamento;
> #### 3. Os artistas com a maior média de popularidade de músicas nos últimos 5 anos.

## Resolução com um ETL

In [ ]:
%
#### Extração dos dados

In [ ]:
#importação da biblioteca Pandas no ambiente do Databricks com Spark
import pyspark.pandas as ps 

In [ ]:
#localização do arquivo
arq_leitura = '/FileStore/tables/spotify_data.csv'

#leitura do arquivo csv
df = ps.read_csv(arq_leitura)

print(df.shape)

df.head(10)

> #### Dados transformado - resultado 1: As músicas mais populares nos últimos 10 anos;

In [0]:
#filtra o dado extraido para ficar apenas com últimos 10 anos
df_res_1 = df[(df['year']>2012) & (df['year']<=2023)]

#ordena por popularidade de forma descendente
df_res_1 = df_res_1.sort_values(by=['popularity'], ascending = False)

print(df_res_1.shape)

df_res_1.head(10)

> #### Dados transformado - resultado 2: A quantidade de músicas lançadas por artista e ano de lançamento;

In [0]:
#garante que os anos estão dentro de uma faixa "normal" e que não é um "bad data"
df_res_2 = df[(df['year']>1900) & (df['year']<=2023)]

#agroupa os dados por ano e artista e faz a contade de músicas
df_res_2 = df_res_2.groupby(['year', 'artists']).agg(quantidade_musicas=('id', 'count'))

#reinicia o index e não exclui o anterior
df_res_2 = df_res_2.reset_index(drop=False)

#ordera por ano e quantidade de música de forma descendente
df_res_2 = df_res_2.sort_values(by=['year', 'quantidade_musicas'], ascending = False)

print(df_res_2.shape)

df_res_2.head(10)

> #### Dados transformado - resultado 3: Os artistas com a maior média de popularidade de músicas nos últimos 5 anos.

In [0]:
#filtra os dados extraidos para garantir apenas os últimos 5 anos
df_res_3 = df[(df['year']>2018) & (df['year']<=2023)]

#agroupa os dados por artistas e agrega pela média de popularidade
df_res_3 = df_res_3.groupby(['artists']).agg(media_popularidade=('popularity', 'avg'))

#reinicia o index sem excluir o anterior
df_res_3 = df_res_3.reset_index(drop=False)

#ordena pela média de popularidade de forma descendente
df_res_3 = df_res_3.sort_values(by=['media_popularidade'], ascending = False)

print(df_res_3.shape)

df_res_3.head(10)

#### Armazenamento dos resultados transformados

In [0]:
#Armazenamento do resultado transformado

#localização do arquivo transformado
arq_escrita = "/FileStore/tables/dados_transformados_aula1_ex1/resultado_1.parquet"

#faz a escrita to arquivo transformado do resultado 1
df_res_1.to_parquet(arq_escrita)

#localização do arquivo transformado
arq_escrita = "/FileStore/tables/dados_transformados_aula1_ex1/resultado_2.parquet"

#faz a escrita to arquivo transformado do resultado 1
df_res_2.to_parquet(arq_escrita)

#localização do arquivo transformado
arq_escrita = "/FileStore/tables/dados_transformados_aula1_ex1/resultado_3.parquet"

#faz a escrita to arquivo transformado do resultado 1
df_res_3.to_parquet(arq_escrita)

In [0]:
dbutils.fs.ls("/FileStore/tables/dados_transformados_aula1_ex1")

## Resolução com um ELT

#### Extração dos dados

In [0]:
#localização do arquivo
arq_leitura = '/FileStore/tables/spotify_data.csv'

#leitura do arquivo csv
df = ps.read_csv(arq_leitura)

print(df.shape)

df.head(10)

In [0]:
df.to_spark().write.format("delta").mode("overwrite").saveAsTable("dados_extraidos_aula1_ex1")

#### Leitura do arquivo armazenado

In [0]:
df = ps.sql("select * from dados_extraidos_aula1_ex1")

In [0]:
#filtra o dado extraido para ficar apenas com últimos 10 anos
df_res_1 = df[(df['year']>2012) & (df['year']<=2023)]

#ordena por popularidade de forma descendente
df_res_1 = df_res_1.sort_values(by=['popularity'], ascending = False)

print(df_res_1.shape)

df_res_1.head(10)

In [0]:
#garante que os anos estão dentro de uma faixa "normal" e que não é um "bad data"
df_res_2 = df[(df['year']>1900) & (df['year']<=2023)]

#agroupa os dados por ano e artista e faz a contade de músicas
df_res_2 = df_res_2.groupby(['year', 'artists']).agg(quantidade_musicas=('id', 'count'))

#reinicia o index e não exclui o anterior
df_res_2 = df_res_2.reset_index(drop=False)

#ordera por ano e quantidade de música de forma descendente
df_res_2 = df_res_2.sort_values(by=['year', 'quantidade_musicas'], ascending = False)

print(df_res_2.shape)

df_res_2.head(10)

In [0]:
#filtra os dados extraidos para garantir apenas os últimos 5 anos
df_res_3 = df[(df['year']>2018) & (df['year']<=2023)]

#agroupa os dados por artistas e agrega pela média de popularidade
df_res_3 = df_res_3.groupby(['artists']).agg(media_popularidade=('popularity', 'avg'))

#reinicia o index sem excluir o anterior
df_res_3 = df_res_3.reset_index(drop=False)

#ordena pela média de popularidade de forma descendente
df_res_3 = df_res_3.sort_values(by=['media_popularidade'], ascending = False)

print(df_res_3.shape)

df_res_3.head(10)